# CellTypist Annotations - A tool for cell type annotation

## All genes, All cells

###  Naive Method

In [ ]:
import scanpy as sc
import celltypist
import os

common_path = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects/adata_matrices"
# common_path = "/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/ur_objects/adata_matrices"
filepath = common_path
output_path = os.path.join(common_path, "cell_annotations")

# Load the CellTypist model
model = celltypist.models.Model.load(model='Healthy_COVID19_PBMC.pkl')

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Dictionary to store annotated data
annotated_adata_dict = {}

# Process each UMI length
for i in range(1, 13):
    print(f"Processing UMI length: {i}")
    
    # Load the h5ad file for the current UMI length
    adata = sc.read_h5ad(filepath + f"/adata_{i}.h5ad")
    
    # Normalize counts per cell to 10,000
    sc.pp.normalize_total(adata, target_sum=1e4)
    
    # Apply log1p transformation
    sc.pp.log1p(adata)
    
    # Check matrix sample (optional)
    print(f"Sample of the matrix for UMI length {i}:")
    print(adata.X[:5, :5])
    print(f"Min: {adata.X.min()}, Max: {adata.X.max()}")
    
    # Annotate the cell types using the CellTypist model
    predictions = celltypist.annotate(adata, model=model, majority_voting=True)
    
    # Store annotated data
    annotated_adata = predictions.to_adata()
    annotated_adata_dict[i] = annotated_adata
    
    # Save annotated AnnData object
    annotated_output_path = os.path.join(output_path, f"annotated_adata_matrices/annotated_adata_{i}.h5ad")
    annotated_adata.write_h5ad(annotated_output_path)
    print(f"Annotated data saved to: {annotated_output_path}\n")

    # # Compute UMAP
    # sc.tl.umap(annotated_adata)

    # # Plot UMAP with majority voting labels
    # sc.pl.umap(
    #     annotated_adata, 
    #     color=['majority_voting'], 
    #     legend_loc='on data', 
    #     title=f"UMAP for UMI length {i}", 
    #     show=True  # Display instead of saving
    # )

###  Methods of Moments Estimator 

In [ ]:
import scanpy as sc
import celltypist
import os


common_path = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects/inv_nonunif_matrices" # 1k
# common_path = "/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/ur_objects/inverted2_adata_matrices" # 10k
# Define file paths
filepath = common_path
output_path = os.path.join(common_path, "cell_annotations")

# Load the CellTypist model
model = celltypist.models.Model.load(model='Healthy_COVID19_PBMC.pkl')

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Dictionary to store annotated data
annotated_adata_dict = {}

# Process each UMI length
for i in range(1, 13):
    print(f"Processing UMI length: {i}")
    
    # Load the h5ad file for the current UMI length
    adata = sc.read_h5ad(filepath + f"/adata_{i}.h5ad")
    
    # Normalize counts per cell to 10,000
    sc.pp.normalize_total(adata, target_sum=1e4)
    
    # Apply log1p transformation
    sc.pp.log1p(adata)
    
    # Check matrix sample (optional)
    print(f"Sample of the matrix for UMI length {i}:")
    print(adata.X[:5, :5])
    print(f"Min: {adata.X.min()}, Max: {adata.X.max()}")
    
    # Annotate the cell types using the CellTypist model
    predictions = celltypist.annotate(adata, model=model, majority_voting=True)
    
    # Store annotated data
    annotated_adata = predictions.to_adata()
    annotated_adata_dict[i] = annotated_adata
    
    # Save annotated AnnData object
    annotated_output_path = os.path.join(output_path, f"annotated_inv_matrices/annotated_adata_{i}.h5ad")
    annotated_adata.write_h5ad(annotated_output_path)
    print(f"Annotated data saved to: {annotated_output_path}\n")

    # Compute UMAP
    # sc.tl.umap(annotated_adata)

    # # Plot UMAP with majority voting labels
    # sc.pl.umap(
    #     annotated_adata, 
    #     color=['majority_voting'], 
    #     legend_loc='on data', 
    #     title=f"UMAP for UMI length {i}(inverted)", 
    #     show=True  # Display instead of saving
    )

### - Celltypist Error Plot  

In [ ]:
import os
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Paths
regular_path = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects/adata_matrices/cell_annotations/annotated_adata_matrices"
inverted_path = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects/inv_nonunif_matrices/cell_annotations/annotated_inv_matrices"

# 1) Load regular annotated adata matrices into a dict
annotated_ads_reg = {
    i: sc.read_h5ad(f"{regular_path}/annotated_adata_{i}.h5ad")
    for i in range(1, 13)
}

# 2) Use UMI 12 as ground truth
ground_truth_adata = annotated_ads_reg[12]
common_cells = sorted(ground_truth_adata.obs_names)
ground_truth = ground_truth_adata.obs.loc[common_cells, 'majority_voting'].astype(str).values

# 3) Compute error vectors
error_vector_reg = np.zeros(12)
error_vector_inv = np.zeros(12)

for i in range(1, 13):
    # Regular predictions aligned to UMI 12
    ad_reg = annotated_ads_reg[i]
    present = [c for c in common_cells if c in ad_reg.obs_names]
    preds_reg = ad_reg.obs.loc[present, 'majority_voting'].astype(str).values
    true_aligned = ground_truth_adata.obs.loc[present, 'majority_voting'].astype(str).values
    error_vector_reg[i - 1] = np.mean(preds_reg != true_aligned)

    # Inverted predictions aligned to UMI 12
    ad_inv = sc.read_h5ad(f"{inverted_path}/annotated_adata_{i}.h5ad")
    present_inv = [c for c in common_cells if c in ad_inv.obs_names]
    preds_inv = ad_inv.obs.loc[present_inv, 'majority_voting'].astype(str).values
    true_aligned_inv = ground_truth_adata.obs.loc[present_inv, 'majority_voting'].astype(str).values
    error_vector_inv[i - 1] = np.mean(preds_inv != true_aligned_inv)

# 4) Plot both error curves
plt.figure(figsize=(10, 10))

# Blue for Naive (Regular)
plt.plot(
    range(1, 13), error_vector_reg, marker='o', linestyle='-',
    label='Naive estimator', markersize=10, linewidth=3, color='#1f77b4'
)

# Green for Collision-aware (Inverted)
plt.plot(
    range(1, 13), error_vector_inv, marker='o', linestyle='-',
    label='Non-uniform Estimator', markersize=10, linewidth=3, color='#2ca02c'
)

plt.xlabel('UMI Length', fontsize=20)
plt.ylabel('Proportion of incorrect predictions', fontsize=20)
plt.ylim(bottom=0)
plt.xticks(range(1, 13), fontsize=12)
plt.axhline(y=0, color='k', linestyle='--', linewidth=1)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.title('Error in Cell Type Predictions (All Genes)', fontsize=24)
plt.legend(fontsize=16)
plt.tight_layout()
# out_file = "/data/dagyeman/cellranger/bam_file_analysis/figures_ur/manuscript_figures/10k_PBMCs/10k_error_plot_consensus.pdf"
# os.makedirs(os.path.dirname(out_file), exist_ok=True)
# plt.savefig(out_file)
plt.show()


## All Cells, All Genes, No MALAT1 

### Naive Method 

In [ ]:
import scanpy as sc
import celltypist
import os
import numpy as np

common_path = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects/adata_matrices"
# common_path = "/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/ur_objects/adata_matrices"
filepath = common_path
output_path = os.path.join(common_path, "cell_annotations")

# Load the CellTypist model
model = celltypist.models.Model.load(model='Healthy_COVID19_PBMC.pkl')

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Dictionary to store annotated data
annotated_adata_dict = {}

# Example remove array (replace with actual list of gene names)
remove = np.array(['MALAT1'])  

# Process each UMI length
for i in range(1, 13):
    print(f"Processing UMI length: {i}")
    
    # Load the h5ad file for the current UMI length
    adata = sc.read_h5ad(filepath + f"/adata_{i}.h5ad")
    
    # Remove genes in the remove list
    adata = adata[:, ~np.in1d(adata.var_names, remove)].copy()
    
    # Normalize counts per cell to 10,000
    sc.pp.normalize_total(adata, target_sum=1e4)
    
    # Apply log1p transformation
    sc.pp.log1p(adata)
    
    # Check matrix sample (optional)
    print(f"Sample of the matrix for UMI length {i}:")
    print(adata.X[:5, :5])
    print(f"Min: {adata.X.min()}, Max: {adata.X.max()}")
    
    # Annotate the cell types using the CellTypist model
    predictions = celltypist.annotate(adata, model=model, majority_voting=True)
    
    # Store annotated data
    annotated_adata = predictions.to_adata()
    annotated_adata_dict[i] = annotated_adata
    
    # Save annotated AnnData object
    annotated_output_path = os.path.join(output_path, f"annotated_adata_matrices/annotated_adata_{i}.h5ad")
    annotated_adata.write_h5ad(annotated_output_path)
    print(f"Annotated data saved to: {annotated_output_path}\n")

    # # Compute UMAP
    # sc.tl.umap(annotated_adata)

    # # Plot UMAP with majority voting labels
    # sc.pl.umap(
    #     annotated_adata, 
    #     color=['majority_voting'], 
    #     legend_loc='on data', 
    #     title=f"UMAP for UMI length {i}", 
    #     show=True  # Display instead of saving
    # )


### Method of Moments Estimator

In [ ]:
import scanpy as sc
import celltypist
import os


common_path = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects/inv_nonunif_matrices" # 1k
# common_path = "/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/ur_objects/inverted2_adata_matrices" # 10k
# Define file paths
filepath = common_path
output_path = os.path.join(common_path, "cell_annotations")

# Load the CellTypist model
model = celltypist.models.Model.load(model='Healthy_COVID19_PBMC.pkl')

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Dictionary to store annotated data
annotated_adata_dict = {}


remove = np.array(['MALAT1'])  

# Process each UMI length
for i in range(1, 13):
    print(f"Processing UMI length: {i}")
    
    # Load the h5ad file for the current UMI length
    adata = sc.read_h5ad(filepath + f"/adata_{i}.h5ad")
    
    adata = adata[:, ~np.in1d(adata.var_names, remove)].copy()
    
    # Normalize counts per cell to 10,000
    sc.pp.normalize_total(adata, target_sum=1e4)
    
    # Apply log1p transformation
    sc.pp.log1p(adata)
    
    # Check matrix sample (optional)
    print(f"Sample of the matrix for UMI length {i}:")
    print(adata.X[:5, :5])
    print(f"Min: {adata.X.min()}, Max: {adata.X.max()}")
    
    # Annotate the cell types using the CellTypist model
    predictions = celltypist.annotate(adata, model=model, majority_voting=True)
    
    # Store annotated data
    annotated_adata = predictions.to_adata()
    annotated_adata_dict[i] = annotated_adata
    
    # Save annotated AnnData object
    annotated_output_path = os.path.join(output_path, f"annotated_inv_matrices/annotated_adata_{i}.h5ad")
    annotated_adata.write_h5ad(annotated_output_path)
    print(f"Annotated data saved to: {annotated_output_path}\n")

    # Compute UMAP
    # sc.tl.umap(annotated_adata)

    # Plot UMAP with majority voting labels
    # sc.pl.umap(
    #     annotated_adata, 
    #     color=['majority_voting'], 
    #     legend_loc='on data', 
    #     title=f"UMAP for UMI length {i}(inverted)", 
    #     show=True  # Display instead of saving
    # )


### Celltypist Error Plot

In [ ]:
import os
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# Paths
regular_path = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects/adata_matrices/cell_annotations/annotated_adata_matrices"
inverted_path = "/data/dagyeman/cellranger/bam_file_analysis/1k_PBMCs/ub_objects/inv_nonunif_matrices/cell_annotations/annotated_inv_matrices"

# regular_path = "/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/ur_objects/adata_matrices/cell_annotations/annotated_adata_matrices"
# inverted_path = "/data/dagyeman/cellranger/bam_file_analysis/10k_PBMCs/ur_objects/inverted2_adata_matrices/cell_annotations/annotated_inv_matrices"

# 1) Load regular annotated adata matrices into a dict
annotated_ads_reg = {
    i: sc.read_h5ad(f"{regular_path}/annotated_adata_{i}.h5ad")
    for i in range(1, 13)
}

# 2) Build consensus ground truth from UMI 8–12
#    a) find common cells across UMI 8–12
cells_8_12 = set(annotated_ads_reg[8].obs_names) 
for i in range(9, 13):
    cells_8_12 &= set(annotated_ads_reg[i].obs_names)
common_cells = sorted(cells_8_12)

#    b) majority‐vote per cell
consensus_labels = {}
for cell in common_cells:
    votes = [annotated_ads_reg[i].obs.loc[cell, 'majority_voting'] for i in range(8, 13)]
    consensus_labels[cell] = Counter(votes).most_common(1)[0][0]

# 3) Prepare ground truth array aligned to common_cells
ground_truth = np.array([consensus_labels[cell] for cell in common_cells], dtype=str)

# 4) Compute error vectors
error_vector_reg = np.zeros(12)
error_vector_inv = np.zeros(12)
print(error_vector_reg)
print(error_vector_inv)

for i in range(1, 13):
    # Regular predictions aligned to consensus cells
    ad_reg = annotated_ads_reg[i]
    present = [c for c in common_cells if c in ad_reg.obs_names]
    preds_reg = ad_reg.obs.loc[present, 'majority_voting'].astype(str).values
    true_aligned = np.array([consensus_labels[c] for c in present], dtype=str)
    error_vector_reg[i - 1] = np.mean(preds_reg != true_aligned)

    # Inverted predictions aligned to consensus cells
    ad_inv = sc.read_h5ad(f"{inverted_path}/annotated_adata_{i}.h5ad")
    present_inv = [c for c in common_cells if c in ad_inv.obs_names]
    preds_inv = ad_inv.obs.loc[present_inv, 'majority_voting'].astype(str).values
    true_aligned_inv = np.array([consensus_labels[c] for c in present_inv], dtype=str)
    error_vector_inv[i - 1] = np.mean(preds_inv != true_aligned_inv)

print(error_vector_reg)
print(error_vector_inv)
print(1 - error_vector_reg[3])

# 5) Plot both error curves
plt.figure(figsize=(10, 10))

# Blue for Naive (Regular)
plt.plot(
    range(1, 13), error_vector_reg, marker='o', linestyle='-',
    label='Naive estimator', markersize=10, linewidth=3, color='#1f77b4'
)

# Green for Collision-aware (Inverted)
plt.plot(
    range(1, 13), error_vector_inv, marker='o', linestyle='-',
    label='Non-uniform Estimator', markersize=10, linewidth=3, color='#2ca02c'
)

plt.xlabel('UMI Length', fontsize=20)
plt.ylabel('Proportion of incorrect predictions', fontsize=20)
plt.ylim(bottom=0)
plt.xticks(range(1, 13), fontsize=12)
plt.axhline(y=0, color='k', linestyle='--', linewidth=1)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.title('Error in Cell Type Predictions (All Genes, No MALAT1)', fontsize=24)
plt.legend(fontsize=16)
plt.tight_layout()
# out_file = "/data/dagyeman/cellranger/bam_file_analysis/figures_ur/manuscript_figures/10k_PBMCs/10k_error_plot_consensus.pdf"
# os.makedirs(os.path.dirname(out_file), exist_ok=True)
# plt.savefig(out_file)
plt.show()